## Data Loading

In [1]:
# imports
from octis.models.CTM import CTM
from octis.dataset.dataset import Dataset
from octis.optimization.optimizer import Optimizer
from skopt.space.space import Real, Categorical, Integer
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.dataset.dataset import Dataset
from skopt.space.space import Real, Categorical, Integer
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.optimization.optimizer import Optimizer
optimizer=Optimizer()
import time
import en_core_web_sm
nlp = en_core_web_sm.load()
dataset = Dataset()
dataset.load_custom_dataset_from_folder("pre_Flower_stop")
docs = [" ".join(words) for words in dataset.get_corpus()]

/sw/arch/RHEL8/EB_production/2022/software/Python/3.10.4-GCCcore-11.3.0/lib/python3.10/site-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
#creating vocab, dataset

# import os
# import string
# from octis.preprocessing.preprocessing import Preprocessing
# os.chdir(os.path.pardir)
# # Initialize preprocessing
# preprocessor = Preprocessing(vocabulary=None, max_features=None, 
#                              remove_punctuation=True, punctuation=string.punctuation,
#                              lemmatize=True, stopword_list='english',
#                              min_chars=1, min_words_docs=0)

# # preprocess
# dataset = preprocessor.preprocess_dataset(documents_path=r'Pats_BERT_LLAMA/NL_tm_map/NL_octis_pre/corpus.tsv')
# # save the preprocessed dataset
# dataset.save('pre_OCT')

## Model

In [4]:
#MODEL CONFIG
from octis.models.NeuralLDA import NeuralLDA
model = NeuralLDA(num_topics=10)

search_space = {"num_layers": Integer(1,3), 
                "num_neurons": Categorical({100, 200, 300}),
                "dropout": Real(0.0, 0.95)}

In [5]:
#SETTING METRICS
coherence = Coherence(texts=dataset.get_corpus(), measure = 'c_v')
diversity = TopicDiversity(topk=10)

In [6]:
#RUN CONFIGURATION
optimization_runs=10
model_runs=2

In [ ]:
#OPTIMIZATION

start = time.time()
optimization_result = optimizer.optimize(
    model, dataset, coherence, search_space, number_of_call=optimization_runs, 
    model_runs=model_runs, save_models=True, 
    extra_metrics=[diversity], # to keep track of other metrics
    save_path='results/test_neuralLDA//')
end = time.time()
duration = end - start
optimization_result.save_to_csv("results_neuralLDA.csv")
print('Optimizing model took: ' + str(round(duration)) + ' seconds.')

Current call:  0
Epoch: [1/100]	Samples: [32364/3236400]	Train Loss: 511.25437467653103	Time: 0:00:07.645674
Epoch: [1/100]	Samples: [6936/693600]	Validation Loss: 477.42883878045706	Time: 0:00:00.959840
Epoch: [2/100]	Samples: [64728/3236400]	Train Loss: 498.5807568667153	Time: 0:00:06.465861
Epoch: [2/100]	Samples: [6936/693600]	Validation Loss: 474.9094208714046	Time: 0:00:00.997591
Epoch: [3/100]	Samples: [97092/3236400]	Train Loss: 497.91070337122267	Time: 0:00:06.449047
Epoch: [3/100]	Samples: [6936/693600]	Validation Loss: 470.68871253694493	Time: 0:00:00.960042
Epoch: [4/100]	Samples: [129456/3236400]	Train Loss: 496.06625773187415	Time: 0:00:06.624415
Epoch: [4/100]	Samples: [6936/693600]	Validation Loss: 472.63261552610476	Time: 0:00:00.963167
Epoch: [5/100]	Samples: [161820/3236400]	Train Loss: 496.64090055211807	Time: 0:00:06.442305
Epoch: [5/100]	Samples: [6936/693600]	Validation Loss: 471.009249606897	Time: 0:00:00.964919
Epoch: [6/100]	Samples: [194184/3236400]	Train Los

In [ ]:
# optimizer=Optimizer()
# optimization_result = optimizer.optimize(
#     model, dataset, npmi, search_space, number_of_call=optimization_runs,
#     model_runs=model_runs, save_models=True,
#     extra_metrics=None, # to keep track of other metrics
#     save_path='results/test_ctm//')

In [3]:
optimizer = Optimizer()
json_path = 'results/test_neuralLDA/result.json'
optimizer.resume_optimization(json_path)

FileNotFoundError: [Errno 2] No such file or directory: 'results/test_neuralLDA/result.json'

## Results

In [ ]:
import json
results = json.load(open("results/test_neuralLDA/result.json",'r'))
results.keys()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.xlabel('Iteration')
plt.ylabel('Coherence score (c_v)')
plt.title('Median coherence score per iteration')
plt.plot(results["f_val"])
plt.show()

In [ ]:
results[ 'f_val'].index(max(results[ 'f_val']))

In [ ]:
print([results['x_iters'][parameter][0] for parameter in results['x_iters'].keys()])